In [1]:
import cv2 as cv
import time
import numpy as np

In [2]:
cap=cv.VideoCapture('test.mp4')

In [3]:
net = cv.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3-tiny.weights')
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)

In [4]:
ln=net.getUnconnectedOutLayersNames()
classes = open('coco.names').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

In [5]:
def post_process(img, outputs, conf):
    H, W = img.shape[:2]

    boxes = []
    confidences = []
    classIDs = []

    for output in outputs:
        scores = output[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > conf:
            x, y, w, h = output[:4] * np.array([W, H, W, H])
            p0 = int(x - w//2), int(y - h//2)
            p1 = int(x + w//2), int(y + h//2)
            boxes.append([*p0, int(w), int(h)])
            confidences.append(float(confidence))
            classIDs.append(classID)
            # cv.rectangle(img, p0, p1, WHITE, 1)

    indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
            cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)


In [6]:
while cap.isOpened():
    sucess,frame=cap.read()
    
    if not sucess:
       
        break
    frame=cv.resize(frame,(416,416))
    frame=cv.flip(frame,1)
    img=frame.copy()
    blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    t0 = time.time()
    outputs = net.forward(ln)
    t = time.time() - t0
    outputs = np.vstack(outputs)
    post_process(img, outputs, 0.5)
    
    
    
    
    
    
    cv.imshow('window',img)
    #cv.displayOverlay('window', f'forward propagation time={t:.3}')
    if cv.waitKey(1)==27:
        break
cap.release()
cv.destroyAllWindows()